# Justin's Brute Force Point-Cloud Alignment (JBFPCA)

Uses Casadi to optimize the transformation matrix T that minizies pointcloud measurement error. Uses a brute force method of feeding in euler angles (psi, theta, phi) and translations (x, y, z) into casadi and seeing what pops out. I'm thinking this will be slow but will produce more accurate results than the linear taylor series expansion as seen in Barfoot Ch. 7. More of a proof-of-concept than anything...

# Problem Definition
Feature location definition:
$ \\ P_k = \begin{bmatrix}  x \\ y \\ z \\ 1 \end{bmatrix} \approx p_{k,0}$

Measurement of feature $k$ at pose $j$:
$ \\ p_{k,j} $

Transformation from pose $j$ to $j+1$:
$ \\ T_{j,j+1} $

Measurement error definition between poses $j$ and $j+1$ for feature $k$:
$ \\ e_{k,j+1} = p_{k,j+1} - T_{j,j+1}*p_{k,j} $

Cost function $J$:
$ \\ J(T_{j,j+1}) = \sum\limits_{k} e_{k,j+1}^T*Q*e_{k,j+1}$

So for pose $0$ to pose $1$:
$ \\ J(T_{0,1}) = \sum\limits_{k} e_{k,1}^T*Q*e_{k,1} = \sum\limits_{k} (p_{k,1} - T_{0,1}*p_{k,0})^T*Q*(p_{k,1} - T_{0,1}*p_{k,0})$

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import casadi as ca
import time
import BF_PCA

In [2]:
# Define symbolic variables used for optimization
angles_sym = ca.vertcat(ca.SX.sym('psi'), ca.SX.sym('theta'), ca.SX.sym('phi'))
trans_sym = ca.vertcat(ca.SX.sym('x'), ca.SX.sym('y'), ca.SX.sym('z'))
all_sym = ca.vertcat(angles_sym, trans_sym)
all_sym

SX([psi, theta, phi, x, y, z])

In [3]:
# Generate test data

# Points observed from pose 0:
points_0 = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 0],
])

R_true = BF_PCA.euler2rot(1, 0.5, -1)
t_true = np.array([[1],
                   [3],
                   [2],
                  ])
T_01_true = np.vstack([np.hstack([R_true, t_true]), np.array([[0, 0, 0, 1]])])
T_01_true

points_1 = BF_PCA.applyT(points_0, T_01_true)

print('T_01_true:', T_01_true)
print('points_0:', points_0)
print('points_1:', points_1)

T_01_true: [[ 0.47415988 -0.67261892 -0.56811636  1.        ]
 [ 0.73846026 -0.0475419   0.67261892  3.        ]
 [-0.47942554 -0.73846026  0.47415988  2.        ]
 [ 0.          0.          0.          1.        ]]
points_0: [[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 1]
 [0 1 1]
 [1 1 0]]
points_1: [[ 1.47415988  3.73846026  1.52057446]
 [ 0.32738108  2.9524581   1.26153974]
 [ 0.43188364  3.67261892  2.47415988]
 [ 0.90604352  4.41107918  1.99473434]
 [-0.24073528  3.62507702  1.73569962]
 [ 0.80154096  3.69091836  0.7821142 ]]


In [4]:
# Build casadi cost function
J = BF_PCA.build_cost(
            points_0=points_0,
            points_1=points_1,
            all_sym=all_sym)

# Setup and run optimizer
# Symbols/expressions
nlp = {}                 # NLP declaration
nlp['x'] = all_sym       # decision vars
nlp['f'] = J      # objective
nlp['g'] = 0             # constraints

# Create solver instance
# opts = {'ipopt.print_level':0, 'print_time':0, 'ipopt.sb': 'yes'}
# F = ca.nlpsol('F','ipopt',nlp,opts);
F = ca.nlpsol('F','ipopt',nlp);

# Solve the problem using a guess
# This uses original landmark/measure association (associates which landmark we think the measurement is measuring)
x_input = np.array([0, 0, 0, 0, 0, 0])
optim = F(x0=x_input)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       17

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [5]:
xhat = np.array(optim['x'])

euler_hat = xhat[0:3]
R_hat = BF_PCA.euler2rot(euler_hat[0][0], euler_hat[1][0], euler_hat[2][0])
t_hat = xhat[3:6]
curr_cost = float(optim['f'])


T_01_hat = np.vstack([np.hstack([R_hat, t_hat]), np.array([[0, 0, 0, 1]])])
T_01_hat

points_1 = BF_PCA.applyT(points_0, T_01_hat)
points_1

array([[ 1.47415988,  3.73846026,  1.52057446],
       [ 0.32738108,  2.9524581 ,  1.26153974],
       [ 0.43188364,  3.67261892,  2.47415988],
       [ 0.90604352,  4.41107918,  1.99473434],
       [-0.24073528,  3.62507702,  1.73569962],
       [ 0.80154096,  3.69091836,  0.7821142 ]])

In [6]:
T_01_true

array([[ 0.47415988, -0.67261892, -0.56811636,  1.        ],
       [ 0.73846026, -0.0475419 ,  0.67261892,  3.        ],
       [-0.47942554, -0.73846026,  0.47415988,  2.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [9]:
T_01_hat

array([[ 0.47415988, -0.67261892, -0.56811636,  1.        ],
       [ 0.73846026, -0.0475419 ,  0.67261892,  3.        ],
       [-0.47942554, -0.73846026,  0.47415988,  2.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])